In [85]:
# Import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import pickle as pkl
import sys

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import *

import torch.utils.data as data
import torch
from torch_geometric.data import Data
from torch.nn import Linear
from torch_geometric.nn import GCNConv
from torch.utils.data import Dataset

In [61]:
path = "C:/Users/gsamp/OneDrive/Documents/AI-3/2n Semestre/Projecte de Síntesi 2/GraphAnomaly/dades_guillem/"
df_classes = pd.read_csv(path + "elliptic_txs_classes.csv") # Nodes' labels
df_edges = pd.read_csv(path + "elliptic_txs_edgelist.csv") # Edges
df_features = pd.read_csv(path + "elliptic_txs_features.csv", header=None) # Nodes' features

# Change column names of df_features
colNames1 = {'0': 'txId', 1: "Time step"}
colNames2 = {str(ii+2): "Local_feature_" + str(ii+1) for ii in range(93)}
colNames3 = {str(ii+95): "Aggregate_feature_" + str(ii+1) for ii in range(72)}

colNames = dict(colNames1, **colNames2, **colNames3 )
colNames = {int(jj): item_kk for jj,item_kk in colNames.items()}

df_features = df_features.rename(columns=colNames)

In [62]:
# Specify the path to your pickle file
pickle_file_path = 'C:\\Users\\gsamp\\OneDrive\\Documents\\AI-3\\2n Semestre\\Projecte de Síntesi 2\\GraphAnomaly\\elipticData_graph.pkl'

# Open the pickle file in binary mode
with open(pickle_file_path, 'rb') as f:
    # Load the data from the pickle file
    G = pkl.load(f)


In [63]:
df_merged = pd.merge(left=df_features, right=df_classes, on="txId", how="left")
df_merged = df_merged.set_index('txId')

class_mapping = {'unknown': 0, '1':1, '2':2}
df_merged['class'] = df_merged['class'].replace(class_mapping)

df_merged = df_merged.reset_index()

C:\Users\gsamp\AppData\Local\Temp\ipykernel_8072\3362275724.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_merged['class'] = df_merged['class'].replace(class_mapping)


In [64]:
#aquest mapping es farà servir per saber de quin node es tracta i poder mirar si hia ha
#errors a l'hora de generar edges i tot això
mapping_txid = dict(zip(df_merged['txId'], list(df_merged.index)))

In [65]:
m = False
if m:
    df_edges = df_edges.replace({'txId1': mapping_txid, 'txId2': mapping_txid})
    df_edges.to_pickle('dades_guillem\mapped_edes.pkl')
else:
    df_edges_mapped = pd.read_pickle('dades_guillem\mapped_edes.pkl')

In [104]:
x = torch.tensor(df_merged.drop(columns=['class', 'Time step']).values, dtype=torch.float)
edge_index = torch.tensor(df_edges_mapped.values, dtype=torch.long).T
y = torch.tensor(df_merged['class'].values)
time = torch.tensor(df_merged['Time step'].values)


data = Data(x=x, edge_index=edge_index, y=y, time=time)


test_size = 0.2  # 20% of the data will be in the test set

# Split data into train and test sets
train_idx, test_idx = train_test_split(range(len(y)), test_size=test_size, random_state=42)

# Create train and test masks
train_mask = torch.zeros(len(y), dtype=torch.bool)
test_mask = torch.zeros(len(y), dtype=torch.bool)

train_mask[train_idx] = True
test_mask[test_idx] = True

data.train_mask = train_mask
data.test_mask = test_mask

In [88]:
#no es fa servir
class GraphDataset(Dataset):
    def __init__(self, x, edge_index, y, time):
        self.x = x
        self.edge_index = edge_index
        self.y = y
        self.time = time

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return {
            'x': self.x[idx],
            'edge_index': self.edge_index[idx],
            'y': self.y[idx],
            'time': self.time[idx]
        }
    

In [98]:
class GCN(torch.nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GCNConv(data.num_features, 4)
        self.conv2 = GCNConv(4, 4)
        self.conv3 = GCNConv(4, 2)
        num_classes = torch.unique(data.y).size(0)
        self.classifier = Linear(2, num_classes)

    def forward(self, x, edge_index):
        h = self.conv1(x, edge_index)
        h = h.tanh()
        h = self.conv2(h, edge_index)
        h = h.tanh()
        h = self.conv3(h, edge_index)
        h = h.tanh()  # Final GNN embedding space.
        
        # Apply a final (linear) classifier.
        out = self.classifier(h)

        return out, h

model = GCN()
print(model)

GCN(
  (conv1): GCNConv(166, 4)
  (conv2): GCNConv(4, 4)
  (conv3): GCNConv(4, 2)
  (classifier): Linear(in_features=2, out_features=3, bias=True)
)


In [105]:
model = GCN()
criterion = torch.nn.CrossEntropyLoss()  #Initialize the CrossEntropyLoss function.
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)  # Initialize the Adam optimizer.

def train(data):
    optimizer.zero_grad()  # Clear gradients.
    out, h = model(data.x, data.edge_index)  # Perform a single forward pass.
    loss = criterion(out[data.train_mask], data.y[data.train_mask])  # Compute the loss solely based on the training nodes.
    loss.backward()  # Derive gradients.
    optimizer.step()  # Update parameters based on gradients.
    return loss, h

for epoch in range(10):
    loss, h = train(data)
    print(f'Epoch: {epoch}, Loss: {loss}')

Epoch: 0, Loss: 1.2812554836273193
Epoch: 1, Loss: 1.2487471103668213
Epoch: 2, Loss: 1.2171159982681274
Epoch: 3, Loss: 1.1865522861480713
Epoch: 4, Loss: 1.156948447227478
Epoch: 5, Loss: 1.1280173063278198
Epoch: 6, Loss: 1.0996087789535522
Epoch: 7, Loss: 1.0717217922210693
Epoch: 8, Loss: 1.0444021224975586
Epoch: 9, Loss: 1.0176987648010254
